In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.constants as sc
import ROOT
import glob
import sys
import os
import tqdm

In [18]:
# neutron separation energy of Cf-252
Sn = 6.172 # MeV (energy of emitted neutron)

activity = 19858367638938. # Bq/g (total decays, alphas and neutrons) [2]

neutron_rate = 2.314e6 # neutrons / sec / ug [1]
neutron_frac = 3.092/100 # fraction of neutrons (remainder alpha decays) [1]

density = 15.1 # g/cm^3 (wikipedia)

Si_density = 2.33 # g/cm^3 (from google)

typical_purity = 1/4800 # typical fraction of Cf source that has active Cf-252 (numbers from [1] - ish: 5 cm x 1 cm radius block contains ~50 mg of Cf-252)

volume = 0.1**3 # volume (cm^3)
active_mass = density*(0.1**3)*typical_purity # grams
total_activity = active_mass*activity # Bq (decays per second)

samplerate = neutron_rate*1e6*density*volume*typical_purity # emission rate (neutrons per second)

distance = 25 # distance between Cf and detector (cm)

thermal_crssctn = 0.171 # barns
det_mass = 1 # small detector mass (g)
mSi = 28*sc.atomic_mass # mass of silicon nucleus (kg)
nflux = samplerate/(4*np.pi*distance**2) # neutrons per cm^2 per sec
capture_rate = thermal_crssctn*1e-24*nflux*(det_mass/1e3)/mSi # captures per second

# [1] https://doi.org/10.1016/S0969-8043(00)00214-1
# [2] https://doi.org/10.1016/j.anucene.2023.109699 
# other: https://www.chemlin.org/isotope/californium-252

def exposure_time(Nprimaries):
    # seconds corresponding to Nprimaries
    return Nprimaries/total_activity

In [32]:
detectors = ['lowmass', 'highmass1', 'highmass2']
def get_rates(*files):
    counts = {det: 0 for det in detectors}
    ncounts = {det: 0 for det in detectors}

    for file in files:
        last_event = -1
        hits = {det: 0 for det in detectors}
        tfile = ROOT.TFile.Open(file, 'READ')
        tree = tfile.Get('tree')
        N = tree.GetEntries()

        for k in tqdm.trange(N):
            tree.GetEntry(k)
            EventNum = int(getattr(tree, 'EventNum'))
            VolName = getattr(tree, 'VolName')
            ProcName = getattr(tree, 'ProcName')
            hits[VolName] += 1


            if ProcName == 'nCapture':
                ParentVol = getattr(tree, 'ParentVol')

                if (ParentVol == VolName) and (getattr(tree, 'PName') != 'gamma'):
                    ncounts[VolName] += 1

            if abs(EventNum - last_event) > 1e-6: # new event
                last_event = EventNum
                for det in detectors:
                    if hits[det] > 0:
                        counts[det] += 1
                
                hits = {det: 0 for det in detectors}

        tfile.Close()

    return counts, ncounts



In [57]:
Nprimaries = 5e5
T = exposure_time(Nprimaries)
files_15_10 = [
    #'../build/test_data/ratesim_20250109_125611_t0.root',
    '../build/test_data/ratesim_20250109_130755_t0.root',
] # 5.5e6 primaries

files_15_5 = [
    '../build/test_data/ratesim_20250109_133224_t0.root'
] # 5e6 primaries

files_10_5 = [
    '../build/test_data/ratesim_20250109_135033_t0.root'
] # 5e6 primaries

files_10_10 = [
    '../build/test_data/ratesim_20250109_140549_t0.root'
] # 5e6 primaries

files_10_15 = [
    '../build/test_data/ratesim_20250109_142831_t0.root'
] # 5e6 primaries

files_15_15 = [
    '../build/test_data/ratesim_20250109_144110_t0.root'
] # 5e6 primaries

files_20_5 = [
    '../build/test_data/ratesim_20250109_150118_t0.root'
] # 5e6 primaries
# takeaways: not enough poly. For the same distance from the source, way more background events (and a fair amount of )

counts, ncounts = get_rates(*files_15_5)
T = exposure_time(5e6)

for det in detectors:
    print(f'{det}: {counts[det]/T/1e3:.4g} kHz ({counts[det]:.3g} counts)')
    print(f'    captures: {ncounts[det]/T:.4g} Hz ({ncounts[det]:.3g} counts)')

100%|██████████| 3140/3140 [00:00<00:00, 16019.52it/s]


lowmass: 0 kHz (0 counts)
    captures: 0 Hz (0 counts)
highmass1: 4.873 kHz (390 counts)
    captures: 474.8 Hz (38 counts)
highmass2: 2.786 kHz (223 counts)
    captures: 124.9 Hz (10 counts)


In [52]:
counts

{'lowmass': 0, 'highmass1': 132, 'highmass2': 66}

In [53]:
ncounts

{'lowmass': 0, 'highmass1': 14, 'highmass2': 5}